In [1]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


from hnmchallenge.models.top_pop import TopPop
from hnmchallenge.evaluation.python_evaluation import map_at_k
from hnmchallenge.constant import *
from hnmchallenge.models.sgmc.sgmc import SGMC
from hnmchallenge.models.ease.ease import EASE
from hnmchallenge.models.itemknn.itemknn import ItemKNN 
from hnmchallenge.data_reader import DataReader
import datetime


In [2]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

In [3]:
dataset = FilterdDataset()
dr = DataReader()

In [5]:
last_month_data = dr.get_filtered_full_data()

In [21]:
sorted_data = last_month_data.sort_values([DEFAULT_USER_COL, "t_dat"]).reset_index(drop=True)

In [22]:
sorted_data

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,0,0,0.008458,2
1,2018-09-20,0,0,0.008458,2
2,2019-08-05,0,0,0.007610,2
3,2019-08-16,0,644,0.039576,2
4,2019-10-06,0,1952,0.030492,2
...,...,...,...,...,...
14361083,2020-09-22,1136205,7580,0.022017,2
14361084,2020-09-22,1136205,1765,0.016932,2
14361085,2020-09-22,1136205,838,0.032186,2
14361086,2020-09-22,1136205,12955,0.022017,2


In [26]:
dict_users = sorted_data.groupby(DEFAULT_USER_COL).indices

In [32]:
from tqdm import tqdm_notebook
import math

In [47]:
PERC = 0.2
train_indices = []
test_indices = []
for k,v in tqdm_notebook(dict_users.items()):
    split_index = math.ceil(PERC*len(v)) if len(v) > 4 else 1
    train_data_indices, test_data_indices = v[:-split_index], v[-split_index:]
    
    train_indices.append(train_data_indices)
    test_indices.append(test_data_indices)
    

/tmp/ipykernel_57647/3901551625.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for k,v in tqdm_notebook(dict_users.items()):


  0%|          | 0/1136206 [00:00<?, ?it/s]

In [48]:
train_data = sorted_data.loc[np.concatenate(train_indices)]
test_data = sorted_data.loc[np.concatenate(test_indices)]

In [59]:
# number of items per user on holdout
np.mean(list(map(len, test_indices)))

2.9974212422747284

In [60]:
# number of items per user on train
np.mean(list(map(len, train_indices)))

9.642089550662467

In [62]:
test_data.reset_index(drop=True).to_feather(dr.get_preprocessed_data_path() / "stratified_holdout_80_20.feather")
train_data.reset_index(drop=True).to_feather(dr.get_preprocessed_data_path() / "stratified_holdin_80_20.feather")

In [64]:
len(train_data[DEFAULT_USER_COL].unique())

962730

In [65]:
len(test_data[DEFAULT_USER_COL].unique())

1136206

In [67]:
# 15 percent of the users have bought only one item
1-(962730/1136206)

0.15268005977789234